In [10]:
import pandas as pd
import numpy as np

In [1]:
train1 = pd.read_csv('../Input_Data/e-SNLI/dataset/esnli_train_1.csv')
train2 = pd.read_csv('../Input_Data/e-SNLI/dataset/esnli_train_2.csv')
train = pd.concat([train1, train2])
train = train[train.notnull().apply(all, axis=1)]
dev = pd.read_csv('../Input_Data/e-SNLI/dataset/esnli_dev.csv')
dev = dev[dev.notnull().apply(all, axis=1)]
test = pd.read_csv('../Input_Data/e-SNLI/dataset/esnli_test.csv')
test = test[test.notnull().apply(all, axis=1)]

dev_prepared = pd.read_csv('../02_Extract_Subphrases/prepared_data/subphrase_vectors_dev.csv', sep=';')
dev_prepared = dev_prepared.drop(columns='Unnamed: 0')
dev = dev.set_index('pairID')
rel_pairIDs = dev_prepared.iloc[:,0]
y_hat = dev.loc[rel_pairIDs].gold_label
dev_prepared = dev_prepared.iloc[:,1:].to_numpy()

NameError: name 'pd' is not defined

In [12]:
BN_dev_small = pd.read_csv("../04_BN_Explanations/BN_explanations_small_model.csv", sep=";", index_col=0)
BN_dev_large = pd.read_csv("../04_BN_Explanations/BN_explanations_large_model.csv", sep=";", index_col=0)

In [44]:
dev = dev.loc[rel_pairIDs]
dev = dev.iloc[BN_dev_large.i]

In [13]:
temp = [None, ] * 10
for i in range(1, 11):
    temp[i - 1] = pd.read_csv("../01_GPT3_Explanations/prepared_data/GPT3_explanations" + str(i) + ".csv", sep=";")
gpt3_dev = pd.concat(temp).set_index("pairID")
gpt3_dev = gpt3_dev.loc[rel_pairIDs]
gpt3_dev = gpt3_dev.iloc[BN_dev_large.i]

In [14]:
dev_subphrases = pd.read_csv('../02_Extract_Subphrases/prepared_data/subphrases_dev.csv', sep=',')
dev_subphrases = dev_subphrases.set_index('pairID')
dev_subphrases = dev_subphrases.loc[rel_pairIDs]
dev_subphrases = dev_subphrases.drop("Unnamed: 0", axis=1)
dev_subphrases = dev_subphrases.iloc[BN_dev_large.i]

In [49]:
dev[["Sentence1", "Sentence2"]]

,Sentence1,Sentence2
pairID,,
435054077.jpg#1r1c,A young woman sits crosslegged beside her purs...,The woman is on the couch with the dogs.
2320420111.jpg#1r1e,A man in a bicycle is passing through a house ...,A man is on a bike.
4687453573.jpg#3r1e,Two Asian women are next to a table that has f...,The women are sitting down to lunch.
3361630301.jpg#4r1n,A shopper buys cat food at a Walmart,A homeless person buys cat food for dinner at ...
264009676.jpg#4r1n,Two men are shopping for orange juice.,two men are getting breakfast
...,...,...
2536781293.jpg#3r1n,Six workers are working on the train tracks.,Six workers are completing repair on a train t...
2169709244.jpg#1r1e,A little girl in a blue dress is sitting on a ...,A girl sits on a vehicle.
3487583433.jpg#0r1n,Here is a picture of someone doing diagnostics...,They are holding a notepad.


In [52]:
evaluation_template = pd.concat((BN_dev_large.rename({"y_hat_BN": "y_hat_BN_large", "BN_expl": "BN_expl_large"}, axis="columns"), BN_dev_small[["y_hat_BN", "BN_expl"]].rename({"y_hat_BN": "y_hat_BN_small", "BN_expl": "BN_expl_small"}, axis="columns"), dev_subphrases.reset_index(drop=True), dev[["Sentence1", "Sentence2"]].reset_index(drop=True)), axis=1)
evaluation_template["y_hat_gpt3"] = gpt3_dev.pred_label.to_list()
evaluation_template["gpt3_expl"] = gpt3_dev.pred_explanation.to_list()

In [53]:
evaluation_template

,i,y,y_hat_BN_large,BN_expl_large,y_hat_BN_small,BN_expl_small,string_subj_s1,string_verb_s1,string_obj_s1,string_loc_s1,string_clo_s1,string_subj_s2,string_verb_s2,string_obj_s2,string_loc_s2,string_clo_s2,Sentence1,Sentence2,y_hat_gpt3,gpt3_expl
0,638,contradiction,contradiction,grass is not the same as couch,contradiction,grass is not the same as couch,A young woman,sits,NaN,grass,NaN,woman,is,NaN,couch,NaN,A young woman sits crosslegged beside her purs...,The woman is on the couch with the dogs.,contradiction,"The woman is sitting on the grass, not on a co..."
1,2703,entailment,entailment,"if the subject of sentence 1 is man, then the ...",neutral,there is no indication that the subject of sen...,man,passing,NaN,bicycle,NaN,man,is,NaN,bike,NaN,A man in a bicycle is passing through a house ...,A man is on a bike.,entailment,A man is on a bike passing through a house wit...
2,1457,neutral,neutral,there is no indication that the subject of sen...,contradiction,has is not the same as sitting,a table,has,food,NaN,NaN,The women,sitting,NaN,NaN,NaN,Two Asian women are next to a table that has f...,The women are sitting down to lunch.,entailment,The presence of food on the table suggests tha...
3,1694,neutral,neutral,there is no indication that the object of sent...,entailment,a shopper is the same as a homeless person and...,A shopper,buys,cat food,NaN,NaN,A homeless person,buys,cat food,NaN,NaN,A shopper buys cat food at a Walmart,A homeless person buys cat food for dinner at ...,neutral,Just because someone is buying cat food at Wal...
4,1613,neutral,neutral,there is no indication that the object of sent...,neutral,there is no indication that the verb of senten...,Two men,shopping,NaN,NaN,NaN,two men,getting,breakfast,NaN,NaN,Two men are shopping for orange juice.,two men are getting breakfast,neutral,Orange juice does not necessarily mean they ar...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3030,2817,neutral,neutral,there is no indication that the verb of senten...,neutral,there is no indication that the verb of senten...,Six workers,working,NaN,train tracks,NaN,Six workers,completing,repair,train track,NaN,Six workers are working on the train tracks.,Six workers are completing repair on a train t...,entailment,Working on the train tracks implies repair.
3031,546,entailment,entailment,little girl is the same as girl and if the sub...,entailment,little girl is the same as girl and if the sub...,little girl,sitting,NaN,NaN,blue dress,girl,sits,NaN,vehicle,NaN,A little girl in a blue dress is sitting on a ...,A girl sits on a vehicle.,entailment,A tractor is a vehicle.
3032,382,neutral,neutral,there is no indication that the subject of sen...,neutral,there is no indication that the subject of sen...,picture of,doing,diagnostics,vehicle,NaN,They,holding,a notepad,NaN,NaN,Here is a picture of someone doing diagnostics...,They are holding a notepad.,neutral,The picture does not provide enough informatio...
3033,2177,neutral,neutral,there is no indication that the subject of sen...,neutral,there is no indication that the subject of sen...,Two young children,cook,NaN,kitchen counter,NaN,They,preparing,dinner for their parents,NaN,NaN,Two young children cook by sitting on the kitc...,They are preparing dinner for their parents.,neutral,It cannot be inferred that the children are co...


In [54]:
evaluation_template.to_csv("evaluation_template.csv", sep=";")